# 1. Import Dependencies

In [3]:
!pip install requests beautifulsoup4 pandas numpy

In [1]:
import json
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# 2. Collect Cirque Colors Product Data

In [2]:
# Request HTML page for Cirque Colors - What's New
r = requests.get("https://www.cirquecolors.com/new/")
soup = BeautifulSoup(r.text, "html.parser")
regex = re.compile(".*product-item card*")
results = soup.find_all("article", {"class": regex})

# Existing Cirque Color color words
colors = ["Reds", "Oranges", "Yellows", "Greens", "Blues", 
          "Violets", "Pinks", "Browns", "Monochrome", "Multicolored"]

# Create list to hold categories from HTML page
categories = []
for category in results:
    data = category.get("data-product-category")
    data_split = re.split("\W+", data)
    categories.extend(data_split)

# Create list to hold best selling Cirque Color colors
cc_colors = []
#for category in categories:
for category in categories:
    for color in colors:
        if category == color:
            cc_colors.append(category)

# Shorten list to 30 elements            
del cc_colors[30:]

# Create json variable to store data in external file
cirque_colors_json = json.dumps(cc_colors)
with open("cirque_best_seller_Jan22.json", "w") as cirque_json:
    cirque_json.write(cirque_colors_json)

# Convert list to numpy array
np_cc_colors = np.array(cc_colors)

# 3. Collect OPI Product Data

In [3]:
# Request HTML page for OPI - What's New
ropi = requests.get("https://www.opi.com/whats-new")
soup_opi = BeautifulSoup(ropi.text, "html.parser")
regex_opi = re.compile("\w")
results_opi = soup_opi.find_all("a",{"class":regex_opi})

# Create list to hold data-color-family-primary extractions
data_color = []
for result in results_opi:
    data_opi = result.get("data-color-family-primary")
    data_color.append(data_opi)

# Create list to hold best-selling OPI colors 
opi_colors = []
for color in data_color:
    if color!= None and color!="":
        opi_colors.append(color)

# Shorten list to 30 elements        
del opi_colors[30:]

# Create json variable to store data in external file
opi_colors_json = json.dumps(opi_colors)
with open("opi_best_seller_Jan22.json", "w") as opi_json:
    opi_json.write(opi_colors_json)

# Convert list to numpy array
np_opi_colors = np.array(opi_colors)

# 3. Populate a Pandas DataFrame

In [4]:
# Create Dataframe with Cirque Colors data
polish_df = pd.DataFrame(np_cc_colors, columns={"Cirque Colors"})

In [5]:
# Add OPI colors data to Dataframe
polish_df["OPI Colors"] = np_opi_colors.tolist()

In [6]:
polish_df

,Cirque Colors,OPI Colors
0,Reds,Purple
1,Violets,Blue
2,Multicolored,Red
3,Greens,Blue
4,Blues,White
5,Multicolored,Blue
6,Blues,Red
7,Violets,Pink
8,Multicolored,Green
9,Greens,Pink
